<a href="https://colab.research.google.com/github/jollyredflames/GAN-COVID-Detection/blob/master/CSC420_Project_COVIDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The following lines outline the efforts we have taken to repurpose the conditional-adversarial model presented in the pix2pix paper as an anomaly detection model to detect the cases of viral pneumonia from the chest X-ray images. 

The original model is aimed at a way of style-transfer between pairs of images. For example, the model can be trained on the pairs of aerial images and maps, so in the deployment, it draws maps from aerial images. 

We have made changes to the original pix2pix model that we discuss here:

* The pix2pix model's Generator utilizes the UNet design where skip connections are shared between the encoder/decoder layers. This makes sense in the context of style transfer since we want the network to be able to utilize the information from the input image to come up with the output. However, in the Anomaly Detection context, we want the network to learn to reconstruct CXR images by itself. This means the network will, in theory, do a good job of reconstructing the images it has seen in training ("normal" cases) and it will do poorly for the category of images it does not encounter in training ("abnormal cases").

* Both pix2pix model and our model utilize convolutional autoencoders. However, the pix2pix model shrinks the images a lot before the autoencoder level. However, we want the network to focus on the "local" features of images. Therefore, the number of layers in our Generator has been reduced to 4 on each side of the autoencoder. Since the Discriminator is also just a simple convolutional neural network, we have also reduced the number of layers too to reflect the changes for the Generator. 

# Related Work

Zhang, J et al [1] has been the inspiration behind the work we have done here. They propose the confidence-aware anomaly detection (CAAD) model that works by a convolutional feature detector model feeding into an anomaly detection module and a confidence prediction module that works together to classify instances of healthy control, viral-pneumonia and non-viral pneumonia. Similar to our proposed model, CAAD utilizes an anomaly detection module that enables it to potentially train in the absence of viral pneumonia CXR cases. We have taken CAAD as the state-of-the-art model with a sweeping 87.47% AUC when the confidence prediction module and the anomaly detection module in it work in tandem. 

# Disscussion 

# Documented Code

## Imports

In [1]:
import torch 
print('torch version: ', torch.__version__)
import torchvision
print('torchvision version: ', torchvision.__version__)


from torch.utils import data
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
import re
import zipfile
import random
import cv2

import torch.nn as nn
import torch.optim as optim
from sklearn.utils import shuffle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics


import glob, inspect, time, math
from numpy import savetxt

from torch.nn import functional as F
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

torch.autograd.set_detect_anomaly(True)

torch version:  1.6.0+cu101
torchvision version:  0.7.0+cu101


## Preparing the Dataset

Our current model utlizes a resized version of [CoronaHack](https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset). To use the CXR images in our model, all images have been resized to 256*256. For the model training purposes we utlized Google Colab. The data has been uploaded to Google Drive as a zip file. Here we download the data and unzip it.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = os.path.join('.', 'drive')
data = os.path.join(data, 'My Drive')
data_folder = os.path.join(data, 'CSC420_Project')
data_zip = os.path.join(data_folder, 'CoronaHack_Resized.zip')

zip_ref = zipfile.ZipFile(data_zip, 'r')
zip_ref.extractall('/content/CoronaHack')
zip_ref.close()

In [4]:
pic_path = '/content/CoronaHack/CoronaHack_Resized'

print("Total number of CXR images: {}".format(len(os.listdir(pic_path))))

Total number of CXR images: 5910


## Model Analytics and Logging

The training of neural networks is a costly procedure. We have designed a custome made system of saving model parameters in between training, test snapshots, and train summeries. 

The block below sets the model analytic paths, create the folders on Google Drive and write the headlines for CSV files that record the test/train snapshots. 

In [5]:
## Change this based on you model name ##
model_name = "f1-AnomalyDetect"
#####################################

def make_dir(path):
    try: os.mkdir(path)
    except: 
      pass


save_path = F"/content/drive/My Drive/CSC420_Project/Models/{model_name}"
param_save_path = save_path + "/params" 
snapshots_save_path = save_path + "/snapshots" 
results_path = save_path + "/results"

OnTrain_summary_path = save_path+"/01-OnTrain-summary.csv"
OnTestsnapshots_summary_path = save_path+"/02-OnTest-snapshots-summary.csv"
    


make_dir(results_path)
make_dir(save_path)
make_dir(param_save_path)
make_dir(snapshots_save_path)

result_list = ["tr_restoring", "pca_latent_ontest", "tsne_latent_ontest"]
for result_name in result_list: make_dir(path=os.path.join(results_path, result_name))

## Run this once for each model. It will re-write the files ##
'''
fcsv = open(OnTrain_summary_path, "w")
fcsv.write("epoch,loss_D,loss_G,loss_D_real,loss_D_fake,loss_G_fake,loss_G_L2\n")
fcsv.close()



fcsv = open(OnTestsnapshots_summary_path, "w")
fcsv.write("epoch,G_Normal_loss_Mean,G_AbNormal_loss_Mean,D_Normal_loss_Mean,D_AbNormal_loss_Mean,G_Normal_loss_SD,G_AbNormal_loss_SD,D_Normal_loss_SD,D_AbNormal_loss_SD,G_AUC,D_AUC\n")
fcsv.close()
'''

## Dataset Class

Managing memory allocation is an important part of training any serious neural network. Unlike smaller datasets like MNIST, it is not possible to load the totality of CoronaHack on to the RAM prior to the training. 

Instead, we have designed and written a custome made dataset class. Out data directory consists of a folder of CXR images, and a CSV file preserving the path and lable information for each image. 

The Dataset object is responsible for loading train/test images in batches, splitting the data into folds (the design supports kfold cross-validation), and normalizing the data.


In [6]:
data_path = os.path.join(os.getcwd(),  'CoronaHack')

data_frame_path = os.path.join(data_path, 'dataset.csv')
images_path = os.path.join(data_path, 'CoronaHack_Resized')


class Dataset(object):

    def __init__(self, normalize=True, normal=['Normal'], ab_normal=['Virus'], n_splits=3, n_exp=0):

        print("\nInitializing Dataset...")

        data_frame = pd.read_csv(data_frame_path)

        self.data_frame = data_frame
        self.normal_labels = normal
        self.ab_normal_labels = ab_normal
        self.normal_df = shuffle(data_frame.loc[data_frame['label'].isin(normal)], random_state=42)
        self.ab_normal_df = shuffle(data_frame.loc[data_frame['label'].isin(ab_normal)], random_state=42)

        kf_normal = KFold(n_splits=n_splits, random_state=42)

        self.trains = []
        self.tests_normals = []
        self.tests_ab_normal = []

        for train_index, test_index in kf_normal.split(self.normal_df):
          self.trains.append(train_index)
          self.tests_normals.append(test_index)

        for train_index, test_index in kf_normal.split(self.ab_normal_df):
          self.tests_ab_normal.append(test_index)

        normal_df_train, normal_df_test = self.normal_df.iloc[self.trains[n_exp]], self.normal_df.iloc[self.tests_normals[n_exp]]
        ab_normal_df_test = self.ab_normal_df.iloc[self.tests_ab_normal[n_exp]]
        
        #train_df, test_df = train_test_split(normal_df, test_size=0.33)

        self.train_df = normal_df_train
        self.test_df = shuffle(pd.concat([ab_normal_df_test, normal_df_test]), random_state=42)
        print("Running experiment number {} out of {} ----  Train Cases: {} / Test Cases: {} Out of which {} are COVID"
              .format(n_exp, n_splits-1, self.train_df.shape[0], self.test_df.shape[0], ab_normal_df_test.shape[0]))


        self.normalize = normalize


        self.num_train, self.num_test = self.train_df.shape[0], self.test_df.shape[0]
        self.idx_train, self.idx_test = 0, 0

        sample_image = cv2.imread(os.path.join(images_path, self.train_df.iloc[0]['cxr_img']))

        #self.height = sample_image.shape[0]
        #self.width = sample_image.shape[1]

        self.height = 256
        self.width = 256
        self.channel = 3


        self.num_class = len(normal) + 1
        self.min_val, self.max_val = sample_image.min(), sample_image.max()

        print("Data Summery")
        print("Number of Training Cases: {}".format(self.num_train))
        print("Total Number of Test Cases: {}, Abnormal Cases: {}".format(self.num_test, ab_normal_df_test.shape[0]))
        print("Shape  Height: %d, Width: %d, Channel: %d" %(self.height, self.width, self.channel))
        print("Value  Min: %.3f, Max: %.3f" %(self.min_val, self.max_val))
        print("Number of classes:  %d" %(self.num_class))
        print("Normalization: %r" %(self.normalize))
        if(self.normalize): print("(from %.3f-%.3f to %.3f-%.3f)" %(self.min_val, self.max_val, 0, 1))


    def reset_idx(self): self.idx_train, self.idx_test = 0, 0

    def next_train(self, batch_size=1):


        start, end = self.idx_train, self.idx_train+batch_size

        terminator = False
        if(end >= self.num_train):
            terminator = True
            self.train_df = shuffle(self.train_df)
            start = 0
            end = batch_size

        self.idx_train = end

        train_images = np.zeros((batch_size, self.height, self.width, self.channel))
        train_labels = np.zeros((batch_size), dtype=int)

        for i in range(start, end):
            img_path = self.train_df.iloc[i]['cxr_img']
            img_path = os.path.join(images_path, img_path)
            train_images[i-start] = cv2.imread(img_path)
            train_labels[i-start] = 0


        train_images = np.ndarray.astype(train_images, np.float32)


        if(self.normalize):
            min_x, max_x = train_images.min(), train_images.max()
            train_images = (train_images - min_x) / (max_x - min_x)

        train_images_torch = torch.from_numpy(np.transpose(train_images, (0, 3, 1, 2)))
        train_labels_torch = torch.from_numpy(train_labels)

        return train_images, train_images_torch, train_labels, train_labels_torch, terminator


    def next_test(self, batch_size=1):

        start, end = self.idx_test, self.idx_test + batch_size

        terminator = False
        if (end >= self.num_test):
            terminator = True
            self.test_df = shuffle(self.test_df)
            start = 0
            end = batch_size

        self.idx_test = end

        test_images = np.zeros((batch_size, self.height, self.width, self.channel))
        test_labels = np.zeros((batch_size), dtype=int)
        for i in range(start, end):
            img_path = self.test_df.iloc[i]['cxr_img']
            img_path = os.path.join(images_path, img_path)
            test_images[i - start] = cv2.resize(cv2.imread(img_path), (256,256), interpolation = cv2.INTER_AREA)

            label = self.test_df.iloc[i]['label']
            test_labels[i - start] = 0
            if(label in self.ab_normal_labels):
                #abnormal is 1
                test_labels[i - start] = 1


        test_images = np.ndarray.astype(test_images, np.float32)

        if(self.normalize):
            min_x, max_x = test_images.min(), test_images.max()
            test_images = (test_images - min_x) / (max_x - min_x)

        test_images_torch = torch.from_numpy(np.transpose(test_images, (0, 3, 1, 2)))
        test_labels_torch = torch.from_numpy(test_labels)

        return test_images, test_images_torch, test_labels, test_labels_torch, terminator

## Neural Networks

### The Generator

In [7]:
import torch
from torch import nn

class Flatten(nn.Module):
    def forward(self, input):
        return input.reshape(input.size(0), -1)
  
class Generator(nn.Module):
    def __init__(self, k_size=4):
        super(Generator, self).__init__()
        self.k_size = k_size
        # enc
        self.enc_bn1 = nn.BatchNorm2d(3)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=self.k_size, stride=2, padding=2)

        self.enc_bn2 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=self.k_size, stride=2, padding=2)

        self.enc_bn3 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=self.k_size, stride=2, padding=2)

        self.enc_bn4 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=self.k_size, stride=2, padding=2)


        self.flat = Flatten()
        self.en_linear1 = nn.Linear(17*17*512, 1024)
        self.en_linear2 = nn.Linear(1024, 128)


        #decode        
        self.dec_linear1 = nn.Linear(128, 1024)
        self.dec_linear2 = nn.Linear(1024,17*17*512)

        self.dec_bn1 = nn.BatchNorm2d(512)
        self.convT1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=self.k_size, stride=2, padding=2, output_padding=1)

        self.dec_bn2 = nn.BatchNorm2d(256)
        self.convT2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=self.k_size, stride=2, padding=2, output_padding=1)

        self.dec_bn3 = nn.BatchNorm2d(128)
        self.convT3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=self.k_size, stride=2, padding=2, output_padding=1)

        self.dec_bn4 = nn.BatchNorm2d(64)
        self.convT4 = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=self.k_size, stride=2, padding=2)



        self.leaky_relu = nn.LeakyReLU(0.2)
        self.maxpool = nn.MaxPool2d(2)


        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)




    def encode(self, x):

      x = self.enc_bn1(x)
      x = self.conv1(x)
      x = self.leaky_relu(x)

      x = self.enc_bn2(x)
      x = self.conv2(x)
      x = self.leaky_relu(x)

      x = self.enc_bn3(x)
      x = self.conv3(x)
      x = self.leaky_relu(x)
      
      x = self.enc_bn4(x)
      x = self.conv4(x)
      x = self.leaky_relu(x)


      x = self.flat(x)
      x = self.en_linear1(x)
      x = self.leaky_relu(x)
      x = self.en_linear2(x)

      return(x)

    def decode(self, x):
      x = self.dec_linear1(x)
      x = self.leaky_relu(x)
      x = self.dec_linear2(x)

      x = x.reshape(x.size(0), 512, 17, 17)  

      x = self.dec_bn1(x)
      x = self.convT1(x)
      x = self.leaky_relu(x)

      x = self.dec_bn2(x)
      x = self.convT2(x)
      x = self.leaky_relu(x)

      x = self.dec_bn3(x)
      x = self.convT3(x)
      x = self.leaky_relu(x)

      x = self.dec_bn4(x)
      x = self.convT4(x)
      x = self.leaky_relu(x)

      return x



    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)

        return x

### The Discriminator

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3*2,  64, 4, stride=2, padding=2)
        self.conv2 = nn.Conv2d( 64, 128, 4, stride=2, padding=2)
        self.conv3 = nn.Conv2d(128, 256, 4, stride=2, padding=2)
        self.conv4 = nn.Conv2d(256, 512, 4, stride=2, padding=2)        
        self.conv_bn1 = nn.BatchNorm2d(64)
        self.conv_bn2 = nn.BatchNorm2d(128)
        self.conv_bn3 = nn.BatchNorm2d(256)        
        self.conv_bn4 = nn.BatchNorm2d(512) 
        self.fc = nn.Linear(512*17*17, 1)
        self.conv_act = nn.LeakyReLU(0.2)
        self.op_act = nn.Sigmoid()
        
    def forward(self, x):
        # enc
        x = self.conv_bn1(self.conv_act(self.conv1(x)))  
        x = self.conv_bn2(self.conv_act(self.conv2(x)))    
        x = self.conv_bn3(self.conv_act(self.conv3(x)))  
        x = self.conv_bn4(self.conv_act(self.conv4(x)))  
        x = self.fc(x.reshape(x.shape[0], -1))             
        return self.op_act(x)
    

## Forward-Pass, Objective Function Computations

This forward-pass, objective function implements what has been proposed in the pix2pix paper.

In [9]:
def forward_n_get_loss(img, G, D, c=100):
    img_hat = G(img) 
    orig_pair = torch.cat([img, img], 1)
    hat_pair = torch.cat([img_hat, img], 1)
    prob_orig, prob_hat = D(orig_pair), D(hat_pair).detach()
    prob_hat_G = prob_hat.clone()

    # loss_D
    loss_D_orig = nn.BCELoss()(prob_orig, torch.ones_like(prob_orig))
    loss_D_hat = nn.BCELoss()(prob_hat, torch.zeros_like(prob_hat))
    loss_D = (loss_D_orig + loss_D_hat) *0.5
    # loss_G
    loss_G_hat = nn.BCELoss()(prob_hat, torch.ones_like(prob_hat_G))
    loss_G_L1 = nn.L1Loss()(img_hat, img)
    loss_G = loss_G_hat + c*loss_G_L1
    return loss_D, loss_G, loss_D_orig, loss_D_hat, loss_G_hat, loss_G_L1


## Utilities 

The following block implements a series of functions that help us with the training, testing, and logging process. 

In [10]:
'''
def gray2rgb(gray):

    rgb = np.ones((gray.shape[0], gray.shape[1], 3)).astype(np.float32)
    rgb[:, :, 0] = gray[:, :, 0]
    rgb[:, :, 1] = gray[:, :, 0]
    rgb[:, :, 2] = gray[:, :, 0]

    return rgb
'''

def dat2canvas(data):

    numd = math.ceil(np.sqrt(data.shape[0]))
    [dn, dh, dw, dc] = data.shape
    canvas = np.ones((dh*numd, dw*numd, dc)).astype(np.float32)

    for y in range(numd):
        for x in range(numd):
            try: tmp = data[x+(y*numd)]
            except: pass
            else: canvas[(y*dh):(y*dh)+256, (x*dw):(x*dw)+256, :] = tmp
    #if(dc == 1):
     #   canvas = gray2rgb(gray=canvas)

    return canvas

def save_img(contents, names=["", "", ""], savename=""):

    num_cont = len(contents)
    plt.figure(figsize=(5*num_cont+2, 5))

    for i in range(num_cont):
        plt.subplot(1,num_cont,i+1)
        plt.title(names[i])
        plt.imshow(dat2canvas(data=contents[i]))

    plt.tight_layout()
    plt.savefig(savename)
    plt.close()

def discrete_cmap(N, base_cmap=None):

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)

    return base.from_list(cmap_name, color_list, N)

# Saves a latent plot, used both for t-sne plotting and PCA plots
def latent_plot(latent, y, n, title, savename=""):

    plt.figure(figsize=(6, 5))
    plt.scatter(latent[:, 0], latent[:, 1], c=y, \
        marker='o', edgecolor='none', cmap=discrete_cmap(n, 'jet'))
    plt.colorbar(ticks=range(n))
    plt.grid()
    plt.tight_layout()
    plt.title(title)
    plt.savefig(savename)
    plt.close()

def save_graph(contents, xlabel, ylabel, savename):

    np.save(savename, np.asarray(contents))
    plt.clf()
    plt.rcParams['font.size'] = 15
    plt.plot(contents, color='blue', linestyle="-", label="loss")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.tight_layout(pad=1, w_pad=1, h_pad=1)
    plt.savefig("%s.png" %(savename))
    plt.close()

def tens2nparr(input):

    input = input.cpu()
    output = input.detach().numpy()
    return output

# Computes Mean Squared Loss between two sets of images. This meassure is used for anomaly detection
def restore_loss_functions(x, x_hat):

    x, x_hat = x.cpu(), x_hat.cpu()
    criterion = torch.nn.MSELoss()
    restore_loss = criterion(x_hat, x)
    return restore_loss

def get_device():
  device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

# Trains the models, save a recounstruction plot at the beginning of each epoch
# It is also capable of loading previously saved model parameters. 
# see *param_save_path* in the first code block
def training(G, D, dataset, epochs, batch_size, load=False, snapshot=True):

    # if a model is loaded, this number reflects the epochs of the previous training
    epoch_offset = 0
    if(load):
      pre_trained_models = os.listdir(param_save_path)

      param_G = os.path.join(param_save_path, pre_trained_models[-2])
      param_D = os.path.join(param_save_path, pre_trained_models[-1])
      print('=======')
      print(param_D)
      print(param_G)
      print('=======')

      G.load_state_dict(torch.load(param_G))
      D.load_state_dict(torch.load(param_D))
      print("model loaded succesffully")

      epoch_offset = int(re.findall(r'\d+', pre_trained_models[-1])[0])

      print("---> Model was previously trained on {}-epochs".format(epoch_offset))


    print("\n<Training to new %d epochs (%d of minibatch size)>" %(epochs, batch_size))

    device = get_device()


    start_time = time.time()

    iteration = 0

    test_sq = 10
    test_size = test_sq**2
    list_recon = []
    restore_error = 0

    loss2npval = lambda loss : np.mean(loss.cpu().data.numpy()).round(4)
    loss_names = ["loss_D", "loss_G", "loss_D_real", "loss_D_fake",  \
                  "loss_G_fake", "loss_G_L1"]
    update_DnG_together = True
    G.train()
    D.train()

    for epoch in range(epoch_offset, epoch_offset + epochs):


        x_tr, x_tr_torch, y_tr, y_tr_torch, _ = dataset.next_train(batch_size=test_size) # Initial batch


        x_restore = G(x_tr_torch.cuda())

        x_restore = np.transpose(tens2nparr(x_restore), (0, 2, 3, 1))


        save_img(contents=[x_tr, x_restore, (x_tr-x_restore)**2], \
            names=["Input\n(x)", "Restoration\n(x to x-hat)", "Difference"], \
            savename=os.path.join(results_path, "tr_restoring", "%d.png" %(epoch)))



        while(True):
            if(iteration % 10 == 0):
                print("->Epoch {}, Training Index {}, Latest RestoreError {}".format(epoch, dataset.idx_train, restore_error))

            x_tr, x_tr_torch, y_tr, y_tr_torch, terminator = dataset.next_train(batch_size)

            x_tr_torch = x_tr_torch.cuda()

            # forward and get loss
            c = 100
            losses = forward_n_get_loss(x_tr_torch, G, D, c=c )

            update_D, update_G = (True, True) if update_DnG_together  \
            else ((epo+batch_No)%2!=0, (epo+batch_No)%2==0  )
            
            # update D
            if update_D:
                optimizer_D.zero_grad()
                losses[0].backward(retain_graph=True)
                optimizer_D.step()
                
            # update G
            if update_G:
                optimizer_G.zero_grad()
                losses[1].backward()
                optimizer_G.step()


            

            x_hat = G(x_tr_torch.to(device))

            restore_error = restore_loss_functions(x=x_tr_torch, x_hat=x_hat)
            list_recon.append(restore_error.item())

            x_hat = np.transpose(tens2nparr(x_hat), (0, 2, 3, 1))


            iteration += 1
            if(terminator): break
                
        loss_vals = map(loss2npval, losses)
        loss_info = dict(zip(loss_names,loss_vals))
        print("[{}] {}, time_cost:" \
              .format( epoch, loss_info))
        
        if(epoch % 5 == 0):
          torch.save(G.state_dict(), param_save_path+"/params-epoch%d-G" %(epoch))
          torch.save(D.state_dict(), param_save_path+"/params-epoch%d-D" %(epoch))

          if(snapshot):
            dataset.reset_idx()
            snapshot_onTest(G, D, dataset, epoch)
            torch.cuda.empty_cache()


    elapsed_time = time.time() - start_time
    print("Elapsed: "+str(elapsed_time))

# This function tests the models on the train set of the dataset and logs a bunch of plots and analytics
def snapshot_onTest(G, D, dataset, epoch):

    G_scores_normal, G_scores_abnormal = [], []
    D_scores_normal, D_scores_abnormal = [],[]
    latent_vectors, labels = [], []
    G_scores = []
    D_scores = []


    snap_fcsv = open(snapshots_save_path+"/01-OnTest-epoch{}-snapshot.csv".format(epoch), "w")

    snap_fcsv.write("class,G_score,D_score\n")


    print("Test SnapShot:")
    while(True):
        x, x_torch, y, y_torch, terminator = dataset.next_test(1) # y_te does not used in this prj.

        x_hat = G(x_torch.cuda())
        x_enc = tens2nparr(G.encode(x_torch.cuda()))[0]
        G_recons_error = restore_loss_functions(x=x_torch, x_hat=x_hat)
        x_G_score = G_recons_error.item()

        pair = torch.cat([x_torch.cuda(), x_hat], 1)
        x_D_score = D(pair.cuda()).item()


        labels.append(y[0])
        latent_vectors.append(x_enc)
        G_scores.append(x_G_score)
        D_scores.append(x_D_score)

        if (y[0] == 0): 
          G_scores_normal.append(x_G_score)
          D_scores_normal.append(x_D_score)
        else:
          G_scores_abnormal.append(x_G_score)
          D_scores_abnormal.append(x_D_score)

        snap_fcsv.write("%d,%.7f,%.7f\n" %(y, x_G_score, x_D_score))

        if(terminator): break


    # Saving the latent plots
    pca = PCA(n_components=2)
    tsne = TSNE(n_components=2)

    latent_vectors, labels = np.array(latent_vectors), np.array(labels)

    pca_features = pca.fit_transform(latent_vectors)
    tsne_features = tsne.fit_transform(latent_vectors)

    latent_plot(latent=pca_features, y=labels, n=dataset.num_class, title='PCA Plot-epoch%d' %(epoch),\
            savename=os.path.join(results_path, 'pca_latent_ontest', '%d.png' %(epoch)))

    latent_plot(latent=tsne_features, y=labels, n=dataset.num_class, title='t-SNE Plot-epoch%d' %(epoch),\
        savename=os.path.join(results_path, 'tsne_latent_ontest', '%d.png' %(epoch)))
    

    
    G_fpr, G_tpr, G_thresholds = metrics.roc_curve(labels, G_scores, pos_label=1)
    G_AUC = metrics.auc(G_fpr, G_tpr)


    D_fpr, D_tpr, D_thresholds = metrics.roc_curve(labels, D_scores, pos_label=1)
    D_AUC = metrics.auc(D_fpr, D_tpr)

    D_ins_fpr, D_ins_tpr, D_ins_thresholds = metrics.roc_curve(labels, [1 - dscore for dscore in D_scores], pos_label=1)
    D_ins_AUC = metrics.auc(D_fpr, D_tpr)
 

    G_scores_normal = np.asarray(G_scores_normal)
    G_scores_abnormal = np.asarray(G_scores_abnormal)

    D_scores_normal = np.asarray(D_scores_normal)
    D_scores_abnormal = np.asarray(D_scores_abnormal)

    G_normal_avg, G_normal_std = np.average(G_scores_normal), np.std(G_scores_normal)
    G_abnormal_avg, G_abnormal_std = np.average(G_scores_abnormal), np.std(G_scores_abnormal)


    D_normal_avg, D_normal_std = np.average(D_scores_normal), np.std(D_scores_normal)
    D_abnormal_avg, D_abnormal_std = np.average(D_scores_abnormal), np.std(D_scores_abnormal)


    print("EPOCH: {}".format(epoch))

    print(" Generator Stats: ")
    print("   G-Noraml  avg: %.5f, std: %.5f" %(G_normal_avg, G_normal_std))
    print("   G-Abnoraml  avg: %.5f, std: %.5f" %(G_abnormal_avg, G_abnormal_std))
    print('   G-AUC for epoch{}: {:.5f}'.format(epoch, G_AUC))


    print(" Discriminator Stats: ")
    print("   D-Noraml  avg: %.5f, std: %.5f" %(D_normal_avg, D_normal_std))
    print("   D-Abnoraml  avg: %.5f, std: %.5f" %(D_abnormal_avg, D_abnormal_std))
    print("   D-AUC for epoch{}: {:.5f}".format(epoch, D_AUC))

    print("   D-inverse-AUC for epoch{}: {:.5f}".format(epoch, D_ins_AUC))

    with open(OnTestsnapshots_summary_path, "a") as fcsv:
      fcsv.write("%d,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f,%.6f\n" \
                 %(epoch, G_normal_avg, G_abnormal_avg, D_normal_avg, D_abnormal_avg, G_normal_std, G_abnormal_std, D_normal_std, D_abnormal_std, G_AUC, D_AUC))
      



## Experiment

### Create instances of our neural networks

In [11]:
# build model 
use_cuda = torch.cuda.is_available()
print('use_cuda:', use_cuda)
if use_cuda:
    G = Generator().cuda()
    D = Discriminator().cuda()
else:
    G = Generator()
    D = Discriminator()  

use_cuda: True


### Initialize the weights and bias terms by random distributoions

In [12]:
# initialize weights
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.ConvTranspose2d:
        m.weight.data.normal_(0, 0.02)
        m.bias.data.normal_(0, 0.02)
G.apply(init_weights)
D.apply(init_weights)
print(G.conv1.weight.data.mean().mean())# close to 0.00
print(G.conv1.weight.data.std().mean())# close to 0.02

tensor(-7.0590e-05, device='cuda:0')
tensor(0.0203, device='cuda:0')


### Wanna know the number of parameters?

In [13]:
num_p = 0
for idx_m, model in enumerate([G, D]):
    for p in model.parameters():
        num_p += p.numel()
    #print(model)
print("The number of parameters: %d" %(num_p))

The number of parameters: 311875658


### Optimizers

In [14]:
from torch.optim import Adam
optimizer_G = Adam(G.parameters(), lr=2e-4, betas=(0.5, 0.999))
optimizer_D = Adam(D.parameters(), lr=1e-4, betas=(0.5, 0.999))

### Training 

In [15]:
# an instance of the dataset object
dataset = Dataset(n_exp=0, n_splits=3)


Initializing Dataset...
Running experiment number 0 out of 2 ----  Train Cases: 1050 / Test Cases: 1045 Out of which 519 are COVID
Data Summery
Number of Training Cases: 1050
Total Number of Test Cases: 1045, Abnormal Cases: 519
Shape  Height: 256, Width: 256, Channel: 3
Value  Min: 0.000, Max: 254.000
Number of classes:  2
Normalization: True
(from 0.000-254.000 to 0.000-1.000)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
# training
training(G, D, dataset, epochs=105, batch_size=30, load=True)


<Training to new 105 epochs (30 of minibatch size)>


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


->Epoch 0, Training Index 100, Latest RestoreError 0


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2617: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorIterator.cpp:918.)
  ret = torch._C._nn.l1_loss(expanded_input, expanded_target, _Reduction.get_enum(reduction))


->Epoch 0, Training Index 400, Latest RestoreError 0.16621609032154083
->Epoch 0, Training Index 700, Latest RestoreError 0.10806794464588165
->Epoch 0, Training Index 1000, Latest RestoreError 0.10599958896636963
[0] {'loss_D': 6.1018, 'loss_G': 22.0033, 'loss_D_real': 0.0, 'loss_D_fake': 12.2036, 'loss_G_fake': 1e-04, 'loss_G_L1': 0.22}, time_cost:
Test SnapShot:
EPOCH: 0
 Generator Stats: 
   G-Noraml  avg: 0.09901, std: 0.02912
   G-Abnoraml  avg: 0.11575, std: 0.05014
   G-AUC for epoch0: 0.57636
 Discriminator Stats: 
   D-Noraml  avg: 0.99990, std: 0.00010
   D-Abnoraml  avg: 0.99983, std: 0.00038
   D-AUC for epoch0: 0.39180
   D-inverse-AUC for epoch0: 0.39180


...

->Epoch 88, Training Index 220, Latest RestoreError 0.0013205393915995955
->Epoch 88, Training Index 520, Latest RestoreError 0.0015341341495513916
->Epoch 88, Training Index 820, Latest RestoreError 0.0014236150309443474
[88] {'loss_D': 15.602, 'loss_G': 2.6876, 'loss_D_real': 0.0, 'loss_D_fake': 31.2039, 'loss_G_fake': 0.0, 'loss_G_L1': 0.0269}, time_cost:


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


->Epoch 89, Training Index 190, Latest RestoreError 0.0011766840470954776
->Epoch 89, Training Index 490, Latest RestoreError 0.0013599726371467113
->Epoch 89, Training Index 790, Latest RestoreError 0.0014226397033780813
[89] {'loss_D': 18.4452, 'loss_G': 3.1104, 'loss_D_real': 0.0, 'loss_D_fake': 36.8903, 'loss_G_fake': 0.0, 'loss_G_L1': 0.0311}, time_cost:


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


->Epoch 90, Training Index 160, Latest RestoreError 0.0013003189815208316
->Epoch 90, Training Index 460, Latest RestoreError 0.0011382517404854298
->Epoch 90, Training Index 760, Latest RestoreError 0.0013160003582015634


# train!
G.load_state_dict(torch.load("model/G_0035"))
D.load_state_dict(torch.load("model/D_0035"))

In [ ]:
!nvidia-smi -l 1

# Works Cited

[1] Zhang , Jianpeng. et al, "Viral Pneumonia Screening on Chest X-ray Images Using Confidence-Aware Anomaly Detection," arXiv:2003.12338, 2020.
